In [5]:
import disparity
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import powerlaw
import matplotlib.pyplot as plt
from stats import graph_stats
import backboning as bb

In [14]:
def backbone_pipepline(filepath, threshold, is_directed = False):
    with open(filepath, "rb") as f:
        G = pickle.load(f)

    edge_data = [
    {'src': u, 'trg': v, 'nij': d['weight']}
    for u, v, d in G.edges(data=True)]

    df = pd.DataFrame(edge_data)

    if is_directed:
        disparity_applied = bb.disparity_filter(df, undirected = False)
    else:
        disparity_applied = bb.disparity_filter(df, undirected = True)

    thresh_applied = bb.thresholding(disparity_applied, threshold=threshold).drop(columns=["score"])

    if is_directed:
        backbone = nx.from_pandas_edgelist(
        thresh_applied,
        source="src",
        target="trg",
        edge_attr="nij",
        create_using=nx.DiGraph())
    else:
        backbone = nx.from_pandas_edgelist(
        thresh_applied,
        source="src",
        target="trg",
        edge_attr="nij",
        create_using=nx.Graph())
    
    return backbone

# Hashtag coocurence

In [43]:
hashtag_path = '../../shared-folder-gald/data/unipartite_og.pkl'

In [44]:
hashtag_backbone = backbone_pipepline(hashtag_path, threshold=0.999, is_directed=True)

Calculating DF score...
/Users/renatasapeta/Desktop/bachelor/Bachelor/filtering/backboning.py:184: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_sum = table.groupby(table["src"]).sum().reset_index()


In [45]:
HC_stats = graph_stats(hashtag_backbone)

Calculating best minimal value for power law fit


Assuming nested distributions


In [46]:
for key, val in HC_stats.items():
    print(key,'-', val)

num_nodes - 39665
num_edges - 478445
density - 0.0003041081451264872
mean_degree - 24.124290936593976
std_degree - 96.52941039059701
global_clustering_coeff - 0.09493576026072778
degree_assortativity - -0.07528934052673011
power_law_alpha - 2.918318315988534
power_law_xmin - 387.0
ks_test_statistic - 0.025464531027676096
best_powerlaw_comparison - power law vs truncated_power_law: R = -0.128, p = 0.612


In [47]:
with open('cooc_network_filtered.pkl', 'wb') as f:
    pickle.dump(hashtag_backbone, f)

# Follow

In [22]:
follow_path = '../../shared-folder-gald/data/follow_graph.pkl'

In [35]:
follow_backbone = backbone_pipepline(follow_path, threshold=0.90, is_directed=True)

Calculating DF score...
/Users/renatasapeta/Desktop/bachelor/Bachelor/filtering/backboning.py:184: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  table_sum = table.groupby(table["src"]).sum().reset_index()


In [36]:
follow_stats = graph_stats(follow_backbone)

Calculating best minimal value for power law fit


/Users/renatasapeta/anaconda3/lib/python3.11/site-packages/powerlaw.py:1615: RuntimeWarning: invalid value encountered in divide
  CDF = CDF/norm
'nan' in fit cumulative distribution values.
Likely underflow or overflow error: the optimal fit for this distribution gives values that are so extreme that we lack the numerical precision to calculate them.
Assuming nested distributions


In [37]:
for key, val in follow_stats.items():
    print(key,'-', val)

num_nodes - 26627
num_edges - 402510
density - 0.0005677387124915953
mean_degree - 30.233221917602435
std_degree - 179.07905239759276
global_clustering_coeff - 0.28544694735273846
degree_assortativity - -0.27833885023649185
power_law_alpha - 2.090055669932052
power_law_xmin - 20.0
ks_test_statistic - 0.01337570266014021
best_powerlaw_comparison - power law vs truncated_power_law: R = -1.382, p = 0.096


In [48]:
with open('follow_filtered.pkl', 'wb') as f:
    pickle.dump(follow_backbone, f)